In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv("time_series_covid19_deaths_US.csv")
df = df.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'], axis = 1)
df2 = df.groupby('Province_State').sum()
pop = df2['Population']
df2 = df2.drop('Population', axis = 1)
df3 = df2
for x in list(df3.columns): 
  df3[x] = df3[x] / pop 
df3 = df3.drop([x for x in df3.index if 'Princess' in x])

L=np.array(df3.values.tolist())

In [ ]:
L.shape

In [ ]:
!pip install tslearn

In [ ]:
from tslearn.clustering import TimeSeriesKMeans

nbclus=10
model = TimeSeriesKMeans(n_clusters=nbclus, metric="dtw", max_iter=10)
prediction = model.fit_predict(L)
 
print(prediction)

In [ ]:
fig,axes=plt.subplots(2,5, figsize=(80,50), sharex=True, sharey=True)

for k in range(len(prediction)):
  i=prediction[k]
  axes[i//5, i%5].plot(L[k], label='%.2f'%k)
  axes[i//5, i%5].legend(loc='right')

plt.show()

# première version

### pas fiable


In [ ]:
from tensorflow import keras

In [ ]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(nbclus, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=L.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 300
batch_size = 32


callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    L,
    prediction,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

In [ ]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(L, prediction)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
pred = model.predict(L)
res = []
for p in pred:
    res.append(p.argmax())
res=np.array(res)
print('evaluation', res)
print('correction', prediction)
print(np.all(res==prediction))


# Deuxième version

In [ ]:
from sklearn.svm import SVC

In [ ]:
cls=SVC().fit(L,prediction)

In [ ]:
cls.predict(L)

In [ ]:
#taux de précision
sum(prediction==cls.predict(L))/len(prediction)

# Troisième version

### plutôt fiable

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(L,prediction)

In [ ]:
clf.predict(L)

In [ ]:
#taux de précision
sum(prediction==clf.predict(L))/len(prediction)